## Imports

In [2]:
#!pip install xgboost
#!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --user
#!pip install torchinfo
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#os.environ["WORLD_SIZE"] = "1"
import torch
#import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
import glob
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
from tqdm import tqdm
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from stable_baselines3 import PPO
#from torchinfo import summary
from matplotlib.pyplot import figure


import torch.nn as nn
import torch.nn.functional as F

print('Cuda available:', torch.cuda.is_available())

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda available: False


In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Team1Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(2, 16))
        self.pool = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=(1, 8))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 8))
        #self.fc1 = nn.Linear(in_features=15360, out_features=1200)  # Fully Connected Layer
        self.fc1 = nn.LazyLinear(out_features=1200)
        self.fc2 = nn.Linear(in_features=1200, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=65)
        self.fc4 = nn.Linear(in_features=65, out_features=self.num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # print(f'after conv1: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = F.relu(self.conv2(x))
        # print(f'after conv2: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = F.relu(self.conv3(x))
        # print(f'after conv3: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = x.reshape(x.shape[0], -1)
        # print(f'after reshaping: {x.shape}')
        x = F.relu(self.fc1(x))
        # print(f'fc1: {x.shape}')
        x = F.relu(self.fc2(x))
        # print(f'fc2: {x.shape}')
        x = F.relu(self.fc3(x))
        # print(f'fc3: {x.shape}')
        #x = F.relu(self.fc4(x))
        x = self.fc4(x)
        # print(f'fc3: {x.shape}')

        return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Team2Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(2, 128, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 256, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(256),
            nn.Conv1d(256, 512, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(512),
            nn.Conv1d(512, 1024, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(1024),
        )

        self.fc1 = nn.LazyLinear(512, dtype=torch.float32)
        self.fc2 = nn.Linear(512, num_classes, dtype=torch.float32)
        # self.fc = nn.Sequential(
        #     nn.LazyLinear(512, dtype=torch.float32),
        #     nn.ReLU(),
        #     nn.Linear(512, num_classes, dtype=torch.float32),
        #     nn.ReLU(),
        # )

    def forward(self, x):
        x = x.squeeze(1)
        for layer in self.conv_layers:
            x = layer(x)
        x = x.flatten(1)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #return F.softmax(x, dim=1)
        return x

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# From headley_modrec.py
class Team3Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(1, 16, (2, 16))
        self.conv2 = nn.Conv2d(16, 8, (1, 8))
        self.conv3 = nn.Conv2d(8, 4, (1, 4))
        #self.fc1 = nn.Linear(3996, 512)
        self.fc1 = nn.LazyLinear(out_features=512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, self.num_classes)

        self.activation = {}

    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.tanh(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [6]:
class Team4Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.re1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.re2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.re3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.re4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.flat = nn.Flatten()
        self.drop1 = nn.Dropout(0.5)
        #self.fc1 = nn.Linear(128 * 256,512)  # I dont exactly know why it is 128x256, but I had to do some debugging and hardcode the required value
        self.fc1 = nn.LazyLinear(out_features=512)

        self.re5 = nn.ReLU()
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.re6 = nn.ReLU()
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(self.re1(self.conv1(x)))
        x = self.pool2(self.re2(self.conv2(x)))
        x = self.pool3(self.re3(self.conv3(x)))
        x = self.pool4(self.re4(self.conv4(x)))
        x = self.flat(x)
        x = self.drop1(x)
        x = self.re5(self.fc1(x))
        x = self.drop2(x)
        x = self.re6(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
sig_types = [['2-ASK', ['ask', 2], 0],
             ['4-ASK', ['ask', 4], 1],
             ['8-ASK', ['ask', 8], 2],
             ['BPSK', ['psk', 2], 3],
             ['QPSK', ['psk', 4], 4],
             ['16-QAM', ['qam', 16], 5],
             ['Tone', ['constant'], 6],
             ['P-FMCW', ['p_fmcw'], 7]]
num_classes = len(sig_types)
sig_names = [i[0] for i in sig_types]

device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using device', device)

team1_model = Team1Model(num_classes)
team1_model.load_state_dict(torch.load('../models/team1_model.pt', map_location=torch.device('cpu')))
team1_model.eval()
team1_model.to(device)
print ("loaded team1")

team2_model = Team2Model(num_classes)
team2_model.load_state_dict(torch.load('../models/team2_model.pt', map_location=torch.device('cpu')))
team2_model.eval()
team2_model.to(device)
print ("loaded team2")
team3_model = Team3Model(num_classes)
team3_model.load_state_dict(torch.load('../models/team3_model.pt', map_location=torch.device('cpu')))
team3_model.eval()
team3_model.to(device)
print ("loaded team3")
team4_model = Team4Model(num_classes)
team4_model.load_state_dict(torch.load('../models/team4_model.pt', map_location=torch.device('cpu')))
team4_model.eval()
team4_model.to(device)
print ("loaded team4")

# # Load regular fused model
# reg_fused_model = xgb.XGBClassifier(tree_method="hist")
# reg_fused_model.load_model('../data/summer_models/baseline_fused_2tl.json')
# print('loaded baseline fused model')

# # Load RL fused model
# rl_fused_model = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2, n_estimators=5)
# rl_fused_model.load_model('../data/summer_models/rl_fused_2tl.json')
# with open('../data/summer_models/rl_feature_idxes_2tl.pkl', 'rb') as f:
#     rl_feature_idxes = pickle.load(f)
# print('loaded RL fused model')

# # Load RFE fused model
# rfe_fused_model = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
# rfe_fused_model.load_model('../data/summer_models/rfe_fused_2tl.json')
# with open('../data/summer_models/rfe_feature_idxes_2tl.pkl', 'rb') as f:
#     rfe_feature_idxes = pickle.load(f)
# print('loaded RFE fused model')

# Get the number of trainable parameters in each of the teams' models
team1_params = sum(p.numel() for p in team1_model.parameters() if p.requires_grad)
team2_params = sum(p.numel() for p in team2_model.parameters() if p.requires_grad)
team3_params = sum(p.numel() for p in team3_model.parameters() if p.requires_grad)
team4_params = sum(p.numel() for p in team4_model.parameters() if p.requires_grad)
print('# trainable params, team 1:', team1_params)
print('# trainable params, team 2:', team2_params)
print('# trainable params, team 3:', team3_params)
print('# trainable params, team 4:', team4_params)

#summary(team1_model, input_size=(1, 1, 2, 2048))


Using device cpu


/opt/conda/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


loaded team1
loaded team2
loaded team3
loaded team4
# trainable params, team 1: 38295641
# trainable params, team 2: 8138888
# trainable params, team 3: 4219748
# trainable params, team 4: 34076168


## Helper functions/loading data

In [17]:
MODELS_OBS_INT = 2048
def load_data(channel_path, batch_size, num_batches, num_train_examples, data_obs_int):
    training_data = np.zeros((num_train_examples, 1, 2, MODELS_OBS_INT), dtype=np.float32)

    last_index = 0
    for k in range(num_batches):
        # This is used if we have a labeldata folder that stores class labels
        label_df = pd.read_csv(f"{channel_path}/labeldata/example_{k + 1}.csv")

        num_nans = 0
        iq_file_name = f"{channel_path}/iqdata/example_{k + 1}.dat"
        iq_data = np.fromfile(iq_file_name, np.csingle)
        iq_data = np.reshape(iq_data, (-1, data_obs_int))  # Turn the IQ data into chunks of (chunk size) x (data_obs_int)
        for j in range(iq_data.shape[0]):
            # Check if the current row contains NaN values
            if np.isnan(np.sum(iq_data[j][:])):    
                num_nans += 1
            else:
                iq_array_norm = iq_data[j][:] / np.max(np.abs(iq_data[j][:]))  # Normalize the observation
                iq_array = np.vstack((iq_array_norm.real, iq_array_norm.imag))  # Separate into 2 subarrays - 1 with only real (in-phase), the other only imaginary (quadrature)

                # Pad the iq array with zeros to meet the observation length requirement
                # This is needed because the CNN models have a fixed input size
                iq_array = np.pad(iq_array, ((0, 0), (0, MODELS_OBS_INT - iq_array[0].size)), mode='constant', constant_values=0)

                training_data[last_index, 0, :, :] = iq_array
            last_index += 1
        
        if num_nans > 0:
            print(f'Found {num_nans} rows containing NaNs in {iq_file_name}')
    return torch.utils.data.DataLoader([training_data[i] for i in range(num_train_examples)], batch_size=batch_size, shuffle=False)


In [40]:
def load_labels(channel_path, batch_size, num_batches, num_train_examples, data_obs_int):
    # for i in range(num_batches):
    #     data = pd.read_csv(os.path.join(channel_path, f'labeldata/example_{i+1}.csv')).iloc[:,0]
    #     tensor_data = torch.tensor(data)
    #     print(tensor_data.shape)
    #     print(torch.nn.functional.one_hot(tensor_data).shape)
    #     print(torch.stack([torch.nn.functional.one_hot(tensor_data)]).shape)
    training_labels = torch.stack([torch.nn.functional.one_hot(torch.tensor(pd.read_csv(os.path.join(channel_path, f'labeldata/example_{i+1}.csv')).iloc[:,0])) for i in range(num_batches)])
    return training_labels

In [41]:
#set directory paths
#change to correct for testing data
validation_dir1 = os.path.join('../data', 'validation', '1',)
validation_iq_files = os.path.join(validation_dir1, 'iqdata', 'example_*.dat')
joined_list = glob.glob(validation_iq_files)
num_batches = len(joined_list)
validation_dir2 = os.path.join('../data', 'validation', '2')
validation_iq_files = os.path.join(validation_dir2, 'iqdata', 'example_*.dat')
validation_dir3 = os.path.join('../data', 'validation', '3')
validation_iq_files = os.path.join(validation_dir3, 'iqdata', 'example_*.dat')
validation_dir4 = os.path.join('../data', 'validation', '4')
validation_iq_files = os.path.join(validation_dir4, 'iqdata', 'example_*.dat')

#num_batches = 6
CHUNK_SIZE = 10
num_validation_examples = num_batches * CHUNK_SIZE
MODELS_OBS_INT = 2048
TEAM1_OBS_INT = 2048
# TEAM2_OBS_INT = 1024
# TEAM3_OBS_INT = 512
# TEAM4_OBS_INT = 256
labels = load_labels(validation_dir1, 1, num_batches, num_validation_examples, MODELS_OBS_INT).numpy()
labels = labels.reshape((labels.shape[0] * labels.shape[1], labels.shape[2]))
labels = np.argmax(labels, axis=1)
team1_validation_dataloader = load_data(validation_dir1, 1, num_batches, num_validation_examples, 2048)
team2_validation_dataloader = load_data(validation_dir2, 1, num_batches, num_validation_examples, 1024)
team3_validation_dataloader = load_data(validation_dir3, 1, num_batches, num_validation_examples, 512)
team4_validation_dataloader = load_data(validation_dir4, 1, num_batches, num_validation_examples, 256)
print ("loaded data")

RuntimeError: stack expects each tensor to be equal size, but got [10, 8] at entry 0 and [10, 7] at entry 9

## Feature extraction

In [13]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [ ]:
# team 1 features
features = {}

selected_layer = getattr(team1_model, "fc3")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))

preds_list = []
feats_list1 = []


# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team1_validation_dataloader)):
    with torch.inference_mode():
        preds = team1_model(inputs.to(device))
    feats_list1.append(features['feats'].cpu().numpy())

feats_list1 = np.concatenate(feats_list1)

features1 = np.array(feats_list1)
features1 = torch.tensor(features1)
features1 = features1.reshape(-1, features1.shape[-1])
print("features size: ", features1.size())
print (type(features1))

In [ ]:
# team 2 features
features = {}

selected_layer = getattr(team2_model, "fc1")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))

preds_list = []
feats_list2 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team2_validation_dataloader)):
    with torch.inference_mode():
        preds = team2_model(inputs.to(device))
    feats_list2.append(features['feats'].cpu().numpy())

feats_list2 = np.concatenate(feats_list2)

features2 = np.array(feats_list2)
features2 = torch.tensor(features2)
features2 = features2.reshape(-1, features2.shape[-1])
print("features size: ", features2.size())
print (type(features2))

In [ ]:
# team 3 features
features = {}

selected_layer = getattr(team3_model, "fc3")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))

preds_list = []
feats_list3 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team3_validation_dataloader)):
    with torch.inference_mode():
        preds = team3_model(inputs.to(device))
    feats_list3.append(features['feats'].cpu().numpy())

feats_list3 = np.concatenate(feats_list3)

features3 = np.array(feats_list3)
features3 = torch.tensor(features3)
features3 = features3.reshape(-1, features3.shape[-1])
print("features size: ", features3.size())
print (type(features3))

In [ ]:
# team 4 features
features = {}

selected_layer = getattr(team4_model, "fc2")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))

preds_list = []
feats_list4 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team4_validation_dataloader)):
    with torch.inference_mode():
        preds = team4_model(inputs.to(device))
    feats_list4.append(features['feats'].cpu().numpy())

feats_list4 = np.concatenate(feats_list4)

features4 = np.array(feats_list4)
features4 = torch.tensor(features4)
features4 = features4.reshape(-1, features4.shape[-1])
print("features size: ", features4.size())
print (type(features4))

In [ ]:
combined_tensor = torch.cat((features1, features2, features3, features4), dim=1)
print(f"Combined_tensor size: {combined_tensor.size()}")

## Get accuracy and confusion matrix for regular fused model

In [ ]:
preds = np.argmax(reg_fused_model.predict(combined_tensor), axis=1)

num_correct = np.sum(preds == labels)
accuracy = num_correct / len(preds)
print('Regular fused model validation accuracy:', accuracy)

conf_mat = np.zeros((num_classes, num_classes))

count = 0
with torch.no_grad():
    feats = combined_tensor
    outputs = reg_fused_model.predict(feats)
    outputs = np.argmax(outputs, axis=1)
            
    for i in range(outputs.shape[0]):
        conf_mat[labels[i], outputs[i]] += 1

# Make each row sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Baseline Fused Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/baseline_confusion_matrix.png')
plt.show()

## Get accuracy and confusion matrix for RL fused model

In [ ]:
preds = np.argmax(rl_fused_model.predict(combined_tensor[:, rl_feature_idxes]), axis=1)

num_correct = np.sum(preds == labels)
accuracy = num_correct / len(preds)
print('RL fused model validation accuracy:', accuracy)

conf_mat = np.zeros((num_classes, num_classes))

count = 0
with torch.no_grad():
    feats = combined_tensor
    outputs = rl_fused_model.predict(feats[:, rl_feature_idxes])
    outputs = np.argmax(outputs, axis=1)
            
    for i in range(outputs.shape[0]):
        conf_mat[labels[i], outputs[i]] += 1

# Make each row sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('RL Fused Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/rl_confusion_matrix.png')
plt.show()

## Get accuracy and confusion matrix for RFE fused model

In [ ]:
preds = np.argmax(rfe_fused_model.predict(combined_tensor[:, rfe_feature_idxes]), axis=1)

num_correct = np.sum(preds == labels)
accuracy = num_correct / len(preds)
print('RFE fused model validation accuracy:', accuracy)

conf_mat = np.zeros((num_classes, num_classes))

count = 0
with torch.no_grad():
    feats = combined_tensor
    outputs = rfe_fused_model.predict(feats[:, rfe_feature_idxes])
    outputs = np.argmax(outputs, axis=1)
            
    for i in range(outputs.shape[0]):
        conf_mat[labels[i], outputs[i]] += 1

# Make each row sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('RFE Fused Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/rfe_confusion_matrix.png')
plt.show()

## Get accuracy and confusion matrices for individual teams' models

In [ ]:
# Team 1
# Get model predictions
outputs = []
with torch.inference_mode():
    for idx, inputs in tqdm(enumerate(team1_validation_dataloader)):
        pred = np.argmax(team1_model(inputs.to(device)).cpu())
        outputs.append(pred)
outputs = np.array(outputs)
print(outputs.shape)
# Create the confusion matrix
conf_mat = np.zeros((num_classes, num_classes))
for i in range(outputs.shape[0]):
    conf_mat[labels[i], outputs[i]] += 1
    

# Make each row in the confusion matrix sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

# Get the overall accuracy
num_correct = np.sum(outputs == labels)
accuracy = num_correct / len(outputs)
print('Team 1 model validation accuracy:', accuracy)

# Set up plot
figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Team 1 Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/team1_confusion_matrix.png')
plt.show()

In [ ]:
# Team 2
# Get model predictions
outputs = []
with torch.inference_mode():
    for idx, inputs in tqdm(enumerate(team2_validation_dataloader)):
        pred = np.argmax(team2_model(inputs.to(device)).cpu())
        outputs.append(pred)
outputs = np.array(outputs)

# Create the confusion matrix
conf_mat = np.zeros((num_classes, num_classes))
for i in range(outputs.shape[0]):
    conf_mat[labels[i], outputs[i]] += 1

# Make each row in the confusion matrix sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

# Get the overall accuracy
num_correct = np.sum(outputs == labels)
accuracy = num_correct / len(outputs)
print('Team 2 model validation accuracy:', accuracy)

# Set up plot
figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Team 2 Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/team2_confusion_matrix.png')
plt.show()

In [ ]:
# Team 3
# Get model predictions
outputs = []
with torch.inference_mode():
    for idx, inputs in tqdm(enumerate(team3_validation_dataloader)):
        pred = np.argmax(team3_model(inputs.to(device)).cpu())
        outputs.append(pred)
outputs = np.array(outputs)

# Create the confusion matrix
conf_mat = np.zeros((num_classes, num_classes))
for i in range(outputs.shape[0]):
    conf_mat[labels[i], outputs[i]] += 1

# Make each row in the confusion matrix sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

# Get the overall accuracy
num_correct = np.sum(outputs == labels)
accuracy = num_correct / len(outputs)
print('Team 3 model validation accuracy:', accuracy)

# Set up plot
figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Team 3 Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/team3_confusion_matrix.png')
plt.show()

In [ ]:
# Team 4
# Get model predictions
outputs = []
with torch.inference_mode():
    for idx, inputs in tqdm(enumerate(team4_validation_dataloader)):
        pred = np.argmax(team4_model(inputs.to(device)).cpu())
        outputs.append(pred)
outputs = np.array(outputs)

# Create the confusion matrix
conf_mat = np.zeros((num_classes, num_classes))
for i in range(outputs.shape[0]):
    conf_mat[labels[i], outputs[i]] += 1

# Make each row in the confusion matrix sum to 1
for i in range(conf_mat.shape[0]):
    conf_mat[i] = conf_mat[i] / np.sum(conf_mat[i])

# Get the overall accuracy
num_correct = np.sum(outputs == labels)
accuracy = num_correct / len(outputs)
print('Team 4 model validation accuracy:', accuracy)

# Set up plot
figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, xticklabels=sig_names, yticklabels=sig_names, fmt='.2f')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Team 4 Model Confusion Matrix')
plt.savefig('../data/fusion_plots/confusion_matrices/team4_confusion_matrix.png')
plt.show()